# Convert input spikes of a simulation campaign from .dat to .h5 format

- Incl. patching the `simulation_config.json` files to point to the new .h5 files

In [1]:
import json
import subprocess
from pathlib import Path
from shutil import copyfile, move

In [2]:
CAMPAIGN_PATH = Path("/Users/pokorny/Data/Simulations/BBP-curated/beta-evoked")
SPIKE_PATH = Path("/Users/pokorny/Data/Simulations/BBP-raw/beta-evoked/57a7ef30-637a-42b2-b042-fceb3f4da4ac")

SIM_CONFIG_NAME = "simulation_config.json"
STIM_NAME = "Stimulus_spikeReplayVPM"
STIM_POPULATION = "VPM"

CONVERTER_TOOL = "/Users/pokorny/JupyterLab/git/libsonatareport/build/tools/converter/spikes_converter"

In [3]:
# Check sub-folders
sim_folders = list(CAMPAIGN_PATH.glob("*[0-9]"))
sim_folder_names = [f.name for f in sim_folders]
sim_configs = [_sim / SIM_CONFIG_NAME for _sim in sim_folders]
num_sims = len(sim_folders)
print(f"Found {num_sims} simulation(s) in campaign")
assert all(str(idx) in sim_folder_names for idx in range(num_sims)), "ERROR: Subfolder names do not match simulation indices!"
assert all(_cfg.is_file() for _cfg in sim_configs), "ERROR: Simulation config(s) not found!"

Found 1 simulation(s) in campaign


In [8]:
# Check spike files
input_spike_files = []
output_spike_files = []
for idx, cfg in enumerate(sim_configs):
    with open(cfg, "r") as f:
        cfg_dict = json.load(f)
    inp_file = cfg_dict["inputs"][STIM_NAME]["spike_file"]
    assert Path(inp_file).suffix.lower() == ".dat", "ERROR: .dat input expected!"
    inp_path = SPIKE_PATH / str(idx) / inp_file
    assert inp_path.is_file(), f"ERROR: Input spike file '{inp_path}' not found!"
    out_file = Path(inp_file).stem + ".h5"
    out_path = CAMPAIGN_PATH / str(idx) / out_file
    assert not out_path.is_file(), f"ERROR: Output spike file '{out_path}' already exists!"
    input_spike_files.append(inp_path)
    output_spike_files.append(out_path)
    print(f"Simulation {idx}: Spike file '{inp_file}'")

Simulation 0: Spike file 'vpm_input.dat'


In [9]:
# Run conversion
for idx, (inp, out) in enumerate(zip(input_spike_files, output_spike_files)):
    print(f"Simulation {idx}: Converting '{inp}' -> '{out}'...")

    # Run conversion
    convert_out = subprocess.check_output(f"{CONVERTER_TOOL} {inp} {STIM_POPULATION}", shell=True, text=True)
    print(convert_out)
    print()

    # Move converted file from currend directory to destination folder
    converted_file = inp.name + ".h5"
    assert Path(converted_file).is_file(), "ERROR: Converted file not found!"
    move(src=converted_file, dst=out)


Simulation 0: Converting '/Users/pokorny/Data/Simulations/BBP-raw/beta-evoked/57a7ef30-637a-42b2-b042-fceb3f4da4ac/0/vpm_input.dat' -> '/Users/pokorny/Data/Simulations/BBP-curated/beta-evoked/0/vpm_input.h5'...
[REPORTS] [info] :: Trying to convert '/Users/pokorny/Data/Simulations/BBP-raw/beta-evoked/57a7ef30-637a-42b2-b042-fceb3f4da4ac/0/vpm_input.dat' binary report...'
[REPORTS] [info] :: File '/Users/pokorny/Data/Simulations/BBP-raw/beta-evoked/57a7ef30-637a-42b2-b042-fceb3f4da4ac/0/vpm_input.dat' successfully converted to 'vpm_input.dat.h5'




In [12]:
# Patch simulation configs
for idx, cfg in enumerate(sim_configs):
    # Create .BAK
    bak_file = cfg.parent / (cfg.stem + ".BAK")
    if not bak_file.is_file():
        # Create only if not existing yet (to avoid overwriting)
        copyfile(src=cfg, dst=bak_file)

    # Patch config
    with open(cfg, "r") as f:
        cfg_dict = json.load(f)
    
    spk_file = cfg_dict["inputs"][STIM_NAME]["spike_file"]
    assert spk_file == input_spike_files[idx].name, "ERROR: Spike file name mismatch!"
    spk_file_new = output_spike_files[idx].name
    cfg_dict["inputs"][STIM_NAME]["spike_file"] = spk_file_new
    
    with open(cfg, "w") as f:
        json.dump(cfg_dict, f, indent=2)
    print(f"Simulation {idx}: Config file '{cfg}' written")

Simulation 0: Config file '/Users/pokorny/Data/Simulations/BBP-curated/beta-evoked/0/simulation_config.json' written
